## Glob paths to my DEMs.

In [5]:
import glob
from concurrent.futures import ThreadPoolExecutor
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
path_list = glob.glob('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/data/watersheds/*')
csv_list = glob.glob('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/data/csvs/*')

In [6]:
path_list = [x.split(sep='/')[-1] for x in path_list]

In [7]:
def make_df(path):
    gdf = gpd.GeoDataFrame(data = pd.read_csv(path, index_col = 0))
    gdf = gdf.drop(columns=['y', 'max_y', 'x', 'max_x', 'rank'])
    return gdf

In [8]:
with ThreadPoolExecutor() as executor:
    results_iter = executor.map(make_df, csv_list)
    results = list(results_iter)

relief = pd.Series()
avg_elev = pd.Series()
for res in results:
    relief = pd.concat([relief, res.Relief], ignore_index=True)
    avg_elev = pd.concat([avg_elev, res.avg_elevation], ignore_index=True)
    print(res)

           Relief  avg_elevation      North       West      South       East
838    293.960205     980.951660  38.990972 -79.323471  38.981805 -79.314305
839    284.999023     742.010254  38.990972 -79.314305  38.981805 -79.305138
840    297.089783     655.424805  38.990972 -79.305138  38.981805 -79.295971
841    347.762360     618.933960  38.990972 -79.295971  38.981805 -79.286805
842    183.379791     537.855896  38.990972 -79.286805  38.981805 -79.277638
...           ...            ...        ...        ...        ...        ...
34666  105.575378     930.669800  38.422639 -79.543471  38.413472 -79.534305
34667   97.906128     971.339417  38.422639 -79.534305  38.413472 -79.525138
34668  228.418457    1063.217651  38.422639 -79.525138  38.413472 -79.515971
34669  278.259827    1144.715576  38.422639 -79.515971  38.413472 -79.506805
35209  222.125122     984.864319  38.413472 -79.570971  38.404305 -79.561805

[1056 rows x 6 columns]
           Relief  avg_elevation      North       W

In [9]:
min_el = avg_elev.describe()['75%']
max_r = relief.describe()['25%']

In [10]:
i=0
for res in results:
    res['relict'] = [True if res.Relief.iloc[i] <= max_r and res.avg_elevation.iloc[i] >= max_r else False for i in range(len(res))]
    res['basin'] = i
    i+=1

In [11]:
for res in results:
    display(res)

,Relief,avg_elevation,North,West,South,East,relict,basin
838,293.960205,980.951660,38.990972,-79.323471,38.981805,-79.314305,False,0
839,284.999023,742.010254,38.990972,-79.314305,38.981805,-79.305138,False,0
840,297.089783,655.424805,38.990972,-79.305138,38.981805,-79.295971,False,0
841,347.762360,618.933960,38.990972,-79.295971,38.981805,-79.286805,False,0
842,183.379791,537.855896,38.990972,-79.286805,38.981805,-79.277638,False,0
...,...,...,...,...,...,...,...,...
34666,105.575378,930.669800,38.422639,-79.543471,38.413472,-79.534305,False,0
34667,97.906128,971.339417,38.422639,-79.534305,38.413472,-79.525138,False,0
34668,228.418457,1063.217651,38.422639,-79.525138,38.413472,-79.515971,False,0
34669,278.259827,1144.715576,38.422639,-79.515971,38.413472,-79.506805,False,0


,Relief,avg_elevation,North,West,South,East,relict,basin
2006,265.865417,780.038330,38.972639,-78.626805,38.963472,-78.617638,False,1
2550,248.790649,758.671082,38.963472,-78.645138,38.954305,-78.635971,False,1
2551,294.226257,726.081299,38.963472,-78.635971,38.954305,-78.626805,False,1
2552,196.216248,738.667847,38.963472,-78.626805,38.954305,-78.617638,False,1
3095,269.643005,696.354004,38.954305,-78.654305,38.945139,-78.645138,False,1
...,...,...,...,...,...,...,...,...
32570,347.273865,803.678589,38.459305,-78.736805,38.450139,-78.727638,False,1
33105,38.532196,422.364502,38.450139,-78.837638,38.440972,-78.828471,True,1
33106,46.393311,432.917114,38.450139,-78.828471,38.440972,-78.819305,True,1
33107,51.860535,439.380066,38.450139,-78.819305,38.440972,-78.810138,True,1


,Relief,avg_elevation,North,West,South,East,relict,basin
22001,256.528503,420.958954,38.633472,-80.524305,38.624305,-80.515138,False,2
22541,235.852661,424.163269,38.624305,-80.579305,38.615139,-80.570138,False,2
22542,194.544647,377.451233,38.624305,-80.570138,38.615139,-80.560971,False,2
22543,271.922211,420.949036,38.624305,-80.560971,38.615139,-80.551805,False,2
22544,258.022675,414.197876,38.624305,-80.551805,38.615139,-80.542638,False,2
...,...,...,...,...,...,...,...,...
40066,210.895569,1041.555054,38.330972,-80.093471,38.321805,-80.084305,False,2
40067,185.629150,1086.167969,38.330972,-80.084305,38.321805,-80.075138,False,2
40610,215.057678,1085.984863,38.321805,-80.111805,38.312639,-80.102638,False,2
40611,104.734802,993.423096,38.321805,-80.102638,38.312639,-80.093471,False,2


,Relief,avg_elevation,North,West,South,East,relict,basin
14063,240.018188,790.778320,38.770972,-78.214305,38.761805,-78.205138,False,3
14064,332.102722,749.564087,38.770972,-78.205138,38.761805,-78.195971,False,3
14065,321.839111,674.369751,38.770972,-78.195971,38.761805,-78.186805,False,3
14606,258.630981,665.356018,38.761805,-78.241805,38.752639,-78.232638,False,3
14607,338.395172,616.256042,38.761805,-78.232638,38.752639,-78.223471,False,3
...,...,...,...,...,...,...,...,...
30451,36.419937,172.515900,38.495972,-78.140971,38.486805,-78.131805,True,3
30994,32.742889,173.703506,38.486805,-78.168471,38.477639,-78.159305,True,3
30995,38.134369,170.330505,38.486805,-78.159305,38.477639,-78.150138,True,3
30996,34.604034,170.426193,38.486805,-78.150138,38.477639,-78.140971,True,3


,Relief,avg_elevation,North,West,South,East,relict,basin
21952,132.634064,346.544312,38.633472,-80.973471,38.624305,-80.964305,False,4
21953,113.197266,326.443298,38.633472,-80.964305,38.624305,-80.955138,False,4
22493,166.011749,356.618896,38.624305,-81.019305,38.615139,-81.010138,False,4
22494,155.914673,363.886322,38.624305,-81.010138,38.615139,-81.000971,False,4
22495,143.225800,367.091187,38.624305,-81.000971,38.615139,-80.991805,False,4
...,...,...,...,...,...,...,...,...
39956,157.994507,451.446167,38.330972,-81.101805,38.321805,-81.092638,False,4
39957,159.663452,473.352234,38.330972,-81.092638,38.321805,-81.083471,False,4
40489,111.162842,409.802399,38.321805,-81.220971,38.312639,-81.211805,False,4
40490,172.980377,431.343933,38.321805,-81.211805,38.312639,-81.202638,False,4


...

,Relief,avg_elevation,North,West,South,East,relict,basin
78189,131.156494,793.280823,37.689305,-80.982638,37.680139,-80.973471,False,18
78733,220.793701,837.965332,37.680139,-81.000971,37.670972,-80.991805,False,18
78734,166.847351,780.964844,37.680139,-80.991805,37.670972,-80.982638,False,18
78735,146.040894,772.060547,37.680139,-80.982638,37.670972,-80.973471,False,18
78736,125.871216,798.036926,37.680139,-80.973471,37.670972,-80.964305,False,18
...,...,...,...,...,...,...,...,...
107629,267.103577,989.225708,37.194305,-81.385971,37.185139,-81.376805,False,18
108171,150.244629,937.061340,37.185139,-81.422638,37.175972,-81.413471,False,18
108172,201.598999,977.842957,37.185139,-81.413471,37.175972,-81.404305,False,18
108173,364.771729,1078.725708,37.185139,-81.404305,37.175972,-81.395138,False,18


,Relief,avg_elevation,North,West,South,East,relict,basin
76122,257.591675,708.788025,37.725972,-79.910138,37.716805,-79.900971,False,19
76666,202.428955,705.991394,37.716805,-79.928471,37.707639,-79.919305,False,19
76667,187.327026,647.414734,37.716805,-79.919305,37.707639,-79.910138,False,19
76668,290.180878,603.697205,37.716805,-79.910138,37.707639,-79.900971,False,19
77212,245.855835,769.693848,37.707639,-79.928471,37.698472,-79.919305,False,19
...,...,...,...,...,...,...,...,...
100638,223.366638,694.170898,37.313472,-80.405138,37.304305,-80.395971,False,19
101180,106.054138,669.993408,37.304305,-80.441805,37.295139,-80.432638,False,19
101181,165.515625,690.411987,37.304305,-80.432638,37.295139,-80.423471,False,19
101724,121.235168,709.581299,37.295139,-80.460138,37.285972,-80.450971,False,19


,Relief,avg_elevation,North,West,South,East,relict,basin
81355,253.530365,556.083801,37.634305,-81.990971,37.625139,-81.981805,False,20
81356,273.338867,535.233521,37.634305,-81.981805,37.625139,-81.972638,False,20
81357,325.194885,559.594482,37.634305,-81.972638,37.625139,-81.963471,False,20
81901,252.115936,568.884888,37.625139,-81.990971,37.615972,-81.981805,False,20
81902,228.456970,482.937469,37.625139,-81.981805,37.615972,-81.972638,False,20
...,...,...,...,...,...,...,...,...
110876,189.843567,716.874390,37.139305,-81.651805,37.130139,-81.642638,False,20
110877,163.479675,652.926514,37.139305,-81.642638,37.130139,-81.633471,False,20
110878,218.225708,685.240479,37.139305,-81.633471,37.130139,-81.624305,False,20
110879,238.048828,708.634399,37.139305,-81.624305,37.130139,-81.615138,False,20


,Relief,avg_elevation,North,West,South,East,relict,basin
90798,251.729797,1090.826904,37.478472,-80.515138,37.469305,-80.505971,False,21
91315,47.087891,618.019043,37.469305,-80.780971,37.460139,-80.771805,True,21
91316,69.441406,634.188477,37.469305,-80.771805,37.460139,-80.762638,True,21
91317,70.541626,629.579712,37.469305,-80.762638,37.460139,-80.753471,True,21
91318,55.930298,617.841431,37.469305,-80.753471,37.460139,-80.744305,True,21
...,...,...,...,...,...,...,...,...
116933,285.421814,987.137573,37.038472,-81.184305,37.029305,-81.175138,False,21
116934,172.008850,919.652466,37.038472,-81.175138,37.029305,-81.165971,False,21
116935,195.635132,934.685486,37.038472,-81.165971,37.029305,-81.156805,False,21
116941,100.595398,811.184814,37.038472,-81.110971,37.029305,-81.101805,False,21


,Relief,avg_elevation,North,West,South,East,relict,basin
88724,379.834351,1004.314758,37.515139,-79.506805,37.505972,-79.497638,False,22
89268,240.437988,1015.712036,37.505972,-79.525138,37.496805,-79.515971,False,22
89269,294.414429,1015.019836,37.505972,-79.515971,37.496805,-79.506805,False,22
89270,286.892883,896.967041,37.505972,-79.506805,37.496805,-79.497638,False,22
89271,311.209656,733.549561,37.505972,-79.497638,37.496805,-79.488471,False,22
...,...,...,...,...,...,...,...,...
118228,49.755646,274.701599,37.020139,-79.323471,37.010972,-79.314305,True,22
118229,60.604141,269.451996,37.020139,-79.314305,37.010972,-79.305138,True,22
118230,42.086823,276.891785,37.020139,-79.305138,37.010972,-79.295971,True,22
118231,46.646790,265.755890,37.020139,-79.295971,37.010972,-79.286805,True,22


In [ ]:
# i=0
# for path in path_list:
#     !python /sciclone/home/ntlewis/Nick-Lewis-Research/working_files/py_scripts/relief.py x x x x watersheds/{path} pngs/output_{i}.png csvs/relief_{i}.csv
#     i+=1